### 1. 欧氏距离

In [1]:
#搜集偏好
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
                'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
                'The Night Listener': 3.0},
        'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
                'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
                'You, Me and Dupree': 3.5},
        'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
                'Superman Returns': 3.5, 'The Night Listener': 4.0},
        'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                'The Night Listener': 4.5, 'Superman Returns': 4.0,
                'You, Me and Dupree': 2.5},
        'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                'You, Me and Dupree': 2.0},
        'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
                'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
        'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

critics['Toby']['Snakes on a Plane'] #Toby对Snakes on a Plane的评分

4.5

In [2]:
from math import sqrt
# 用“+1后的倒数”来作为相似度指标
def sim_distance(prefs,person1,person2):
    si={} # Get the list of shared_items，就是两个人都打了分的电影
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    # if they have no ratings in common, return 0
    if len(si)==0: return 0 #就当是，没有一部电影是两人都看过的
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
                        for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares) #避免分母为零；返回值∈(0,1]，越大表明越接近

In [3]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

### 2. 皮尔逊相关系数
数据不是很规范（比如某人给出的评分总是相对于平均值大很多）的时候，用皮尔逊相关系数来判断相似度更好。

In [4]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):        
        si={}# 找大家都评过的电影
        for item in prefs[p1]:
                if item in prefs[p2]: si[item]=1

        n=len(si)# Find the number of elements        
        if n==0: return 0 # if they are no ratings in common, return 0

        # Add up all the preferences 求评分之和
        sum1=sum([prefs[p1][it] for it in si])
        sum2=sum([prefs[p2][it] for it in si])

        # Sum up the squares 平方和
        sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
        sum2Sq=sum([pow(prefs[p2][it],2) for it in si])

        # Sum up the products 乘积和
        pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])

        # Calculate Pearson score 皮尔逊相关度的公式
        num=pSum-(sum1*sum2/n)
        den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
        if den==0: return 0
        
        r=num/den
        return r

In [5]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

### 3.找出具有相似品味的人

In [6]:
# 找出最接近的n个
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other)
            for other in prefs if other!=person] #见下文
    scores.sort( )
    scores.reverse( )
    return scores[0:n] #排序后返回前n个

tmp_d={'a':1,'b':2};'a' in tmp_d #对于字典，可以用in检查"某个键是否存在"

True

In [7]:
topMatches(critics,'Toby',n=3) #与Toby相似度最高的3个（默认用皮尔逊相关系数评价）

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

### 4. 根据打分推荐影片

In [8]:
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        if other==person: continue #不用和自己做比较
        sim=similarity(prefs,person,other) #计算相似度（后面用作权重）
        
        if sim<=0: continue # 忽略不想关或者负相关的人
        for item in prefs[other]:
            # other看过，本person没看过的
            if item not in prefs[person] or prefs[person][item]==0: #为什么这里要有or==0？
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim #权重乘分数，求和
                simSums.setdefault(item,0)
                simSums[item]+=sim #权重之和
                
    rankings=[(total/simSums[item],item) for item,total in totals.items( )] #(估分,片名)
    
    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [9]:
getRecommendations(critics,'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [10]:
getRecommendations(critics,'Toby',similarity=sim_distance) #结果可见这儿不管用哪种，估分是差不多的

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

### 5. 找类似的影片
和计算人与人之间的相似度是一样的——只要把原来的输入中的“人”和“影片”对调位子

In [11]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

movies=transformPrefs(critics)
movies #键为片名，值为各影评人的打分

{'Just My Luck': {'Claudia Puig': 3.0,
  'Gene Seymour': 1.5,
  'Lisa Rose': 3.0,
  'Mick LaSalle': 2.0},
 'Lady in the Water': {'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 2.5,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0},
 'Snakes on a Plane': {'Claudia Puig': 3.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 4.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.0,
  'Mick LaSalle': 4.0,
  'Toby': 4.5},
 'Superman Returns': {'Claudia Puig': 4.0,
  'Gene Seymour': 5.0,
  'Jack Matthews': 5.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.5,
  'Mick LaSalle': 3.0,
  'Toby': 4.0},
 'The Night Listener': {'Claudia Puig': 4.5,
  'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 3.0,
  'Michael Phillips': 4.0,
  'Mick LaSalle': 3.0},
 'You, Me and Dupree': {'Claudia Puig': 2.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 3.5,
  'Lisa Rose': 2.5,
  'Mick LaSalle': 2.0,
  'Toby': 1.0}}

In [12]:
#可直接用原先的topMatches
topMatches(movies,'Superman Returns')
#有负数说明“负相关”，即如倾向于喜欢“Superman Returns”的人一般不喜欢“The Night Listener”

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

### 6. 有可能喜欢某部影片的人

In [13]:
getRecommendations(movies,'Just My Luck') #按照加权平均评分的高低

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

### 7.基于物品的过滤
以上是基于用户的协作型过滤，使用每一位用户的全部评分来构造数据集（将一个用户和其他所有用户比较）。在数据量大的情况下，基于物品的协作型过滤更高，而且这种方式可以将大量计算任务预先完成。

总体思路：1.为每件物品计算其相似物品（可以预先完成）；2.向某人进行推荐时，查看他评过分的物品，选出排位靠前者，用以构造一个加权列表 which 包含着与“靠前者”相似的物品。

In [14]:
def calculateSimilarItems(prefs,n=10):
    result={} #找出最相似的物品    
    itemPrefs=transformPrefs(prefs) #转成：键为片名，值为各影评人的打分
    c=0
    for item in itemPrefs:
        c+=1
        if c%100==0: print(("%d / %d") % (c,len(itemPrefs))) #用来显示下当前处理进度(针对数据量很大的情况)
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

itemsim=calculateSimilarItems(critics) #找出与每部电影最相似的其他(最多)10部

In [15]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    
    for (item,rating) in userRatings.items( ): #这个人打过分的物品(影片)和评分
        for (similarity,item2) in itemMatch[item]: #与打过分的物品相似的物品
            if item2 in userRatings: continue #如果这个人已经打过分了（已经看过这部电影了），处理另一个相似物品
            
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating #加权求和
            
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity #权重(即相似度)之和
            
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]#每个相似物品的评分

    rankings.sort( )
    rankings.reverse( )
    return rankings #返回逆序列表（从高到低）

In [16]:
getRecommendedItems(critics, itemsim, 'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]